# Importing Libraries

In [1]:
import numpy as np
import cv2 as cv
import os
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

The self made dataset consists of total 9k images with 3k per class

# Haar Cascade classifier

In [4]:
haar_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
p = []
DIR = r'C:\Users\varad\TY_Sem1\Computer Vision\Labs\CoursePROJECT\Updated_dataset'
for i in os.listdir(DIR):
    p.append(i)


In [2]:

face_recognizer = cv.face.LBPHFaceRecognizer_create()


In [ ]:

face_recognizer.read('model_cp_cv_2.yml')

In [3]:
import random

In [6]:
test_x = []
actual_labels = []

for person in p:
    path = os.path.join(DIR, person)
    label = p.index(person)
    random.shuffle(os.listdir(path))
    for img in os.listdir(path)[-600:]:
        img_path = os.path.join(path, img)
        img = cv.imread(img_path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        face_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=2)
        for (x, y, w, h) in face_rect:
                faces_roi = gray[y:y+h, x:x+w]
                test_x.append(faces_roi)
                actual_labels.append(label)

In [7]:
len(actual_labels)

2668

In [8]:
actual_labels_y = np.array(actual_labels)

In [9]:
actual_labels_y[1]

0

In [10]:
len(test_x)

2668

In [11]:
type(test_x[1])

numpy.ndarray

In [12]:
test_X = np.array(test_x)

<ipython-input-12-9163ce59e647>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_X = np.array(test_x)


In [13]:
test_X = test_X/255

In [17]:
le = LabelEncoder()


In [14]:
prediction = []
conf = []
for img in range(0,len(test_x)):
    (predictions,confs ) =face_recognizer.predict(test_X[img])            
    prediction.append(predictions)
    
    conf.append(confs)
prediction[:5]
conf[:5]

[0.0,
 12.799959311283066,
 20.780494407773812,
 20.88756819890774,
 66.27876540044254]

In [15]:
min(conf)

0.0

In [16]:
predicted_value_y = np.array(prediction)

In [17]:
prediction[1]

0

In [18]:
accuracy_score(actual_labels_y, predicted_value_y)

0.8242128935532234

In [19]:
print(classification_report(actual_labels_y, predicted_value_y,target_names=["Kapil", "Sarang", "Varad"]))

              precision    recall  f1-score   support

       Kapil       0.86      0.80      0.83      1235
      Sarang       0.98      0.87      0.93       562
       Varad       0.71      0.83      0.76       871

    accuracy                           0.82      2668
   macro avg       0.85      0.83      0.84      2668
weighted avg       0.84      0.82      0.83      2668



In [37]:
print('Precision: %.3f' % precision_score(actual_labels_y, predicted_value_y, average = 'micro'))

Precision: 0.824


# MTCNN Face Extraction

In [4]:
from mtcnn.mtcnn import MTCNN
import cv2 as cv
import PIL
import os
from PIL import Image
import numpy as np
from numpy import asarray

In [5]:
face_detector = MTCNN()


In [8]:
new_data = []
NEW_DIR = r'C:\Users\varad\TY_Sem1\Computer Vision\Labs\CoursePROJECT\updated_images'
for i in os.listdir(NEW_DIR):
    new_data.append(i)


In [9]:
new_data

['kapil', 'sarang', 'varad']

In [6]:
def face_selection(image):
    faces = face_detector.detect_faces(image)
    if faces == []:
        return image
    else:
        x1, y1, width, height = faces[0]['box']
        x1, y1 = abs(x1), abs(y1)
        cropped_face = image[y1:y1+height,x1:x1+width]
        return cropped_face

In [43]:
train_x = []
labels = []

In [44]:
for person in new_data:
    path = os.path.join(NEW_DIR, person)
    label = new_data.index(person)
    for img in os.listdir(path)[:2400]:
        img_path = os.path.join(path, img)
        img = cv.imread(img_path)
        img = cv.resize(img, (380,270))
        extracted_face = face_selection(img)
        extracted_face = cv.cvtColor(extracted_face, cv.COLOR_BGR2GRAY)        
        train_x.append(extracted_face)
        labels.append(label)

In [45]:
features = np.array(train_x , dtype='object')
labels = np.array(labels)
features = features/255
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.train(features, labels)

face_recognizer.save('model_sdp_mtcnn.yml')

np.save('features_sdp_mtcnn_2.npy',features)
np.save('labels_sdp_mtcnn_2.npy',labels)

The Model is trained on 7200 images i.e. 80% training data 

In [46]:
len(train_x)

7200

In [25]:
face_recognizer = cv.face.LBPHFaceRecognizer_create()

face_recognizer.read('model_sdp_mtcnn.yml')


In [47]:
test_x_mtcnn = []
actual_labels_mtcnn = []

for person in new_data:
    path = os.path.join(NEW_DIR, person)
    label = new_data.index(person)
    for img in os.listdir(path)[-600:]:
        img_path = os.path.join(path, img)
        img = cv.imread(img_path)
        img = cv.resize(img, (380,270))
        extracted_face = face_selection(img)
        extracted_face = cv.cvtColor(extracted_face, cv.COLOR_BGR2GRAY)        
        test_x_mtcnn.append(extracted_face)
        actual_labels_mtcnn.append(label)

In [48]:
len(test_x_mtcnn)

1800

In [49]:
test_x_mtcnn = np.array(test_x_mtcnn, dtype='object')

In [50]:
actual_labels_mtcnn = np.array(actual_labels_mtcnn)

In [51]:
test_x_mtcnn = test_x_mtcnn/255

In [52]:
prediction = []
conf = []
for img in range(0,len(test_x_mtcnn)):
    (predictions,confs ) =face_recognizer.predict(test_x_mtcnn[img])            
    prediction.append(predictions)
    
    conf.append(confs)
prediction[:5]
conf[:5]

[0.0, 0.0, 30.17016856259279, 29.367540468719604, 30.108383657873294]

In [53]:
predicted_value_y = np.array(prediction)

In [54]:
accuracy_score(actual_labels_mtcnn, predicted_value_y)

0.9861111111111112

Accuracy score is 98.611 for 1,800 images from test data

In [56]:
from sklearn.metrics import classification_report

In [57]:
print(classification_report(actual_labels_mtcnn, predicted_value_y,target_names=["Kapil", "Sarang", "Varad"]))

              precision    recall  f1-score   support

       Kapil       0.98      0.98      0.98       600
      Sarang       1.00      0.99      1.00       600
       Varad       0.97      0.98      0.98       600

    accuracy                           0.99      1800
   macro avg       0.99      0.99      0.99      1800
weighted avg       0.99      0.99      0.99      1800



# Text to speech conversion

In [10]:
labels_names = {"Kapil":0, "Sarang": 1, "Varad": 2}

In [9]:
import speech_recognition as sr
from gtts import gTTS
import os

import playsound
import pyttsx3

In [11]:
def speech_to_text():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Recording...")
        audio = r.listen(source)
    data = ""
    try:
        data = r.recognize_google(audio)
        print("You said: " + data)
    except sr.UnknownValueError:
        print("Google Speech Recognition did not understand audio")
    except sr.RequestError as e:
        print("Request Failed; {0}".format(e))
    return data

In [12]:
import time
from itertools import count
from multiprocessing import Process

Pyttsx is used for text to speech conversion

In [13]:
engine = pyttsx3.init()


In [14]:
def text_to_speech_2(original_text):
    engine.say(original_text)
    engine.runAndWait()

In [15]:
text_to_speech_2("Hi Varad !")

# RecordTracking

Creating the .csv file for tracking the person's visit

In [16]:
from datetime import datetime
import csv
import pandas as pd

In [17]:
fields = ['Name', 'Timestamp']
filename = "record.csv"


In [18]:
fields_2 = ['Name', 'Timestamp','Initial Info','Recent visit speech']
filename_2 = "record_2.csv"


In [9]:
rows_2 = [['Ben Affleck', datetime.now().strftime("%H:%M"), 'He is an actor', 'He spoke nicely'],
         ['Elton John', datetime.now().strftime("%H:%M"), 'He is the famous singer', 'He sang a song'],
         ['Jerry Seinfield', datetime.now().strftime("%H:%M"), 'He is a well known person', 'He visited you']]

In [20]:
rows = [['Ben Affleck', datetime.now().strftime("%H:%M")],
         ['Elton John', datetime.now().strftime("%H:%M")],
         ['Jerry Seinfield', datetime.now().strftime("%H:%M")]]

In [10]:
rows_2 = pd.DataFrame(rows_2, columns=fields_2)


In [ ]:
rows = pd.DataFrame(rows, columns=fields)


In [22]:
rows.to_csv(filename, index=False)

In [11]:
rows_2.to_csv(filename_2, index=False)

In [14]:
record = pd.read_csv(r"record.csv", header=None)

In [15]:
record_2 = pd.read_csv(r"record_2.csv", header=None)

In [16]:
record

,0,1
0,Name,Timestamp
1,Ben Affleck,19:27
2,Elton John,19:27
3,Jerry Seinfield,19:27
4,Varad,19:27
5,Varad,19:28
6,Varad,19:28
7,Varad,19:28
8,Varad,19:28
9,Varad,19:28


In [13]:
record_2

,0,1,2,3
0,Name,Timestamp,Initial Info,Recent visit speech
1,Ben Affleck,19:57,He is an actor,He spoke nicely
2,Elton John,19:57,He is the famous singer,He sang a song
3,Jerry Seinfield,19:57,He is a well known person,He visited you


# Real Time face recognition through Haar Cascade and LBPH

In [16]:
web_cam_vid = cv.VideoCapture(0)
def Video_webcam(web_cam_vid):
    
    frame_width = int(web_cam_vid.get(3))
    frame_height = int(web_cam_vid.get(4))
    while True:
        ret, frame = web_cam_vid.read()
        if ret == True:
        
            gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            faces = haar_cascade.detectMultiScale(gray,1.3,5)
            

                
            for(x,y,w,h) in faces:
                        roi_color = frame[y:y+h,x:x+w]
                        grayscale_roi = cv.cvtColor(roi_color,cv.COLOR_BGR2GRAY)
                        frame_img = np.array(grayscale_roi)
                        frame_img = frame_img/255
                        (prediction, probability) = face_recognizer.predict(frame_img)
                        face_name = list(labels_names.keys())[list(labels_names.values()).index(prediction)]
                        print(probability)
                        print(face_name)
                        visit_info = [[face_name, datetime.now().strftime("%I:%M %p")]]
                        with open(filename, 'a', newline='') as csvfile:
                            
                                csvwriter = csv.writer(csvfile)
                                csvwriter.writerows(visit_info)
                                csvfile.close()
                        text_to_speech_2(str(face_name))
                        time.sleep(8)
                        if int(probability) < 24:
                            print("unknown person")
                            
                        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        cv.putText(frame, "{}".format(face_name), (x - 10, y - 10),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
            cv.imshow('Frame',frame)
            if cv.waitKey(40) & 0xFF == ord('q'):
                break
        else:
            break

    web_cam_vid.release()
    cv.destroyAllWindows()

In [26]:
Video_webcam(web_cam_vid)

26.853365152681707
Varad
26.942030546324105
Varad


In [20]:
web_cam_vid.release()


# MTCNN on LBPH Real time

In [19]:
def bounding_boxes_lms(image):
    faces = face_detector.detect_faces(image)
    if faces == []:
        return None
    bounding_box = faces[0]['box']
    keypoints = faces[0]['keypoints']
    cv.rectangle(image, (bounding_box[0], bounding_box[1]), (bounding_box[0]+bounding_box[2], bounding_box[1]+bounding_box[3]),(0,255,0),2)
    image_info = [image, bounding_box[0], bounding_box[1]]
    return image_info
   


In [20]:
def talk_while_vid():
    input=sr.Recognizer()
    with sr.Microphone() as source:
        audio=input.listen(source)
        data=""
        try:
            data=input.recognize_google(audio)
            
        except sr.UnknownValueError:
            print("Sorry I did not hear your question, Please repeat again.")
        return data

In [21]:
file_2 = open('record_2.csv')
csvreader_2 = csv.reader(file_2)
rows_2 = []
for row in csvreader_2:
        rows_2.append(row)


In [22]:
print(rows_2[-1:][0][0], rows_2[-1:][0][1], rows_2[-1:][0][2])
person_name = rows_2[-1:][0][0]
timing_of_visit = rows_2[-1:][0][1]
initial_info = rows_2[-1:][0][1]

Varad 01:49 AM 


In [23]:
web_cam_vid = cv.VideoCapture(0)
def Video_webcam_mtcnn(web_cam_vid):
    
    frame_width = int(web_cam_vid.get(3))
    frame_height = int(web_cam_vid.get(4))
    while True:
        ret, frame = web_cam_vid.read()
        if ret == True:
            img = cv.resize(frame, (380,270))
            extracted_face = face_selection(img)
            gray = cv.cvtColor(extracted_face,cv.COLOR_BGR2GRAY)
            frame_img = np.array(gray)
            frame_img = frame_img/255
            (prediction, probability) = face_recognizer.predict(frame_img)
            face_name = list(labels_names.keys())[list(labels_names.values()).index(prediction)]
            print(probability,face_name)
            if int(probability) < 24:
                    print("unknown person")
#             visited_person_speech = talk_while_vid()
#             visit_info = [[face_name, datetime.now().strftime("%I:%M %p"),"", visited_person_speech]]
            visit_info = [[face_name, datetime.now().strftime("%I:%M %p")]]

#             with open(filename_2, 'a', newline='') as csvfile:                
#                         csvwriter = csv.writer(csvfile)
#                         csvwriter.writerows(visit_info)
#                         csvfile.close()
            with open(filename, 'a', newline='') as csvfile:                
                        csvwriter = csv.writer(csvfile)
                        csvwriter.writerows(visit_info)
                        csvfile.close()
            text_to_speech_2(str(face_name))
            time.sleep(2)
            
            frame_box = bounding_boxes_lms(frame)[0]
            cv.putText(frame, "{}".format(face_name), (10, 10),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
            cv.imshow('Frame',frame_box)
            if cv.waitKey(35) & 0xFF == ord('q'):
                break
        else:
            break

    web_cam_vid.release()
    cv.destroyAllWindows()

Once the confidence interval is above a threshold value model recognizes the face as known

In [26]:
Video_webcam_mtcnn(web_cam_vid)

29.53314048465977 Varad
69.44736851552634 Varad
55.173111967615256 Varad


In [25]:
from tensorflow.keras.models import load_model


# Record Check

In [32]:
record = pd.read_csv(r"record.csv", header=None)

In [33]:
record_2 = pd.read_csv(r"record_2.csv", header=None)

In [34]:
record_2

,0,1,2,3
0,Name,Timestamp,Initial Info,Recent visit speech
1,Ben Affleck,19:57,He is an actor,He spoke nicely
2,Elton John,19:57,He is the famous singer,He sang a song
3,Jerry Seinfield,19:57,He is a well known person,He visited you
4,Varad,01:48 AM,NaN,hi hi
5,Varad,01:49 AM,NaN,NaN
6,Varad,01:49 AM,NaN,NaN


In [35]:
record

,0,1
0,Name,Timestamp
1,Ben Affleck,19:27
2,Elton John,19:27
3,Jerry Seinfield,19:27
4,Varad,19:27
...,...,...
56,Varad,10:34 AM
57,Varad,10:34 AM
58,Varad,10:34 AM
59,Varad,10:34 AM


# Query Based

In [40]:
file = open('record.csv')

In [41]:
csvreader = csv.reader(file)

In [42]:
rows = []
for row in csvreader:
        rows.append(row)

    

In [43]:
print(rows[-1:][0][0], rows[-1:][0][1])
person_name = rows[-1:][0][0]
timing_of_visit = rows[-1:][0][1]

Varad 10:34 AM


In [36]:
def talk():
    input=sr.Recognizer()
    with sr.Microphone() as source:
        audio=input.listen(source)
        data=""
        try:
            data=input.recognize_google(audio)
            print(data)
            
        except sr.UnknownValueError:
            print("Sorry I did not hear your question, Please repeat again.")
        return data

In [37]:
def create_train():
    for person in p:
        path = os.path.join(DIR, person)
        label = p.index(person)

        for img in os.listdir(path)[:2400]:
            img_path = os.path.join(path, img)
            img_array = cv.imread(img_path)
            gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)
            face_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=2)

            for (x, y, w, h) in face_rect:
                faces_roi = gray[y:y+h, x:x+w]
                features.append(faces_roi)
                labels.append(label)

In [30]:
def known_face_log(web_cam_vid, vistor_name):
    num = 0
    frame_width = int(web_cam_vid.get(3))
    frame_height = int(web_cam_vid.get(4))
    while True:
        ret, frame = web_cam_vid.read()
        if ret == True:       
            gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            faces = haar_cascade.detectMultiScale(gray,1.3,5)
            for(x,y,w,h) in faces:
                        roi_color = frame[y:y+h,x:x+w]
                        grayscale_roi = cv.cvtColor(roi_color,cv.COLOR_BGR2GRAY)
                        frame_img = np.array(grayscale_roi)
                        frame_img = frame_img/255
                           
                        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        cv.putText(frame, "{}".format(face_name), (x - 10, y - 10),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        cv.imwrite(path+f"{path}"+"_" +str(num), roi_color )
            cv.imshow('Frame',frame)
            num +=1
            if cv.waitKey(40) & 0xFF == ord('q'):
                break
        else:
            break

    web_cam_vid.release()
    cv.destroyAllWindows()

In [38]:
import time

In [44]:
while(1):
        text=talk().lower()

        if text==0:
            continue
            
        if "stop" in str(text) or "exit" in str(text) or "bye" in str(text):
            text_to_speech_2("Ok bye and take care")
            break
        
                  
        elif 'time' in text:
            current_time = datetime.now().strftime("%I:%M %p")
            text_to_speech_2(f"the time is {current_time}")     
        
            
        elif 'visited' in text:
            text_to_speech_2(f"{person_name} visited you at {timing_of_visit}")

        else:
            text_to_speech_2("Sorry sir, query not understood")

visited me today who visited me today
show exit


In [34]:
path_dir = r"C:\Users\varad\TY_Sem1\Computer Vision\Labs\CoursePROJECT\Updated_dataset\Positive Images"